In [44]:
import json
from collections import defaultdict, Counter

In [45]:
file_names = ["test", "dev", "train"]
coref_files = [f"../Raw/Coref/{file_name}.jsonlines" for file_name in file_names]
files = [f"../Raw/{file_name}.jsonl" for file_name in file_names]
data = defaultdict(list)
for file, name in zip(files, file_names):
    for line in open(file).readlines():
        data[name].append(json.loads(line))

In [46]:
counts = defaultdict(Counter)
for file in file_names:
    list_of_list = [[event['event_type'] for event in example['event_mentions']] for example in data[file]]
    for lst in list_of_list:
        counts['event_type'].update(lst)
        counts['event_type_level0'].update([x.split(".")[0] for x in lst])
        counts['event_type_level1'].update([".".join(x.split(".")[:2]) for x in lst])
        counts[f'event_type_{file}'].update(lst)
        counts[f'event_type_level0_{file}'].update([x.split(".")[0] for x in lst])
        counts[f'event_type_level1_{file}'].update([".".join(x.split(".")[:2]) for x in lst])
    list_of_list = [[[arg['role'] for arg in event['arguments']] for event in example['event_mentions']] for example in
                    data[file]]
    for lst in list_of_list:
        for args in lst:
            counts['role_type'].update(args)
            counts[f'role_type_{file}'].update(args)

In [47]:
counts['event_type']

Counter({'Life.Die.Unspecified': 525,
         'Conflict.Attack.DetonateExplode': 823,
         'Cognitive.IdentifyCategorize.Unspecified': 60,
         'Life.Injure.Unspecified': 301,
         'Contact.Contact.Correspondence': 18,
         'Movement.Transportation.Unspecified': 156,
         'Contact.Contact.Meet': 53,
         'GenericCrime.GenericCrime.GenericCrime': 75,
         'Justice.Sentence.Unspecified': 61,
         'ArtifactExistence.DamageDestroyDisableDismantle.Damage': 48,
         'Conflict.Attack.Unspecified': 411,
         'Justice.ChargeIndict.Unspecified': 104,
         'ArtifactExistence.DamageDestroyDisableDismantle.Destroy': 51,
         'ArtifactExistence.ManufactureAssemble.Unspecified': 69,
         'Cognitive.Inspection.SensoryObserve': 37,
         'Cognitive.Research.Unspecified': 11,
         'Justice.ArrestJailDetain.Unspecified': 188,
         'Contact.Contact.Unspecified': 202,
         'Contact.ThreatenCoerce.Unspecified': 26,
         'Contact.Contact

In [48]:
counts['event_type_level0']

Counter({'Life': 830,
         'Conflict': 1276,
         'Cognitive': 115,
         'Contact': 543,
         'Movement': 218,
         'GenericCrime': 75,
         'Justice': 557,
         'ArtifactExistence': 195,
         'Medical': 23,
         'Transaction': 70,
         'Personnel': 10,
         'Disaster': 25,
         'Control': 14})

In [49]:
counts['event_type_level1']

Counter({'Life.Die': 525,
         'Conflict.Attack': 1234,
         'Cognitive.IdentifyCategorize': 60,
         'Life.Injure': 301,
         'Contact.Contact': 468,
         'Movement.Transportation': 218,
         'GenericCrime.GenericCrime': 75,
         'Justice.Sentence': 61,
         'ArtifactExistence.DamageDestroyDisableDismantle': 126,
         'Justice.ChargeIndict': 104,
         'ArtifactExistence.ManufactureAssemble': 69,
         'Cognitive.Inspection': 37,
         'Cognitive.Research': 11,
         'Justice.ArrestJailDetain': 188,
         'Contact.ThreatenCoerce': 34,
         'Medical.Intervention': 23,
         'Justice.TrialHearing': 41,
         'Transaction.ExchangeBuySell': 69,
         'Personnel.StartPosition': 4,
         'Contact.RequestCommand': 41,
         'Justice.InvestigateCrime': 92,
         'Disaster.Crash': 10,
         'Control.ImpedeInterfereWith': 14,
         'Conflict.Demonstrate': 37,
         'Disaster.DiseaseOutbreak': 15,
         'Personn

In [50]:
print(len(set(counts['event_type_level1_test'].keys())))
print(len(set(counts['event_type_level1_dev'].keys())))
print(len(set(counts['event_type_level1_train'].keys())))
print("Test/Dev Event types are not in training:",
      set(counts['event_type_level1_test'].keys()).union(set(counts['event_type_level1_dev'].keys())).difference(
          set(counts['event_type_level1_train'].keys())))
print("Training Event types are not in Test/Dev:", set(counts['event_type_level1_train'].keys()).difference(
    set(counts['event_type_level1_test'].keys()).union(set(counts['event_type_level1_dev'].keys()))))

chosen_event_types = set(counts['event_type_level1_train'].keys()).intersection(
    set(counts['event_type_level1_test'].keys()).union(set(counts['event_type_level1_dev'].keys())))
print(len(chosen_event_types))

25
25
33
Test/Dev Event types are not in training: set()
Training Event types are not in Test/Dev: {'Cognitive.TeachingTrainingLearning', 'Justice.Acquit', 'Conflict.Defeat', 'Transaction.Donation', 'Life.Infect'}
28


In [51]:
print(len(set(counts['event_type_level0_test'].keys())))
print(len(set(counts['event_type_level0_dev'].keys())))
print(len(set(counts['event_type_level0_train'].keys())))
set(counts['event_type_level0_test'].keys()).union(set(counts['event_type_level0_dev'].keys())).difference(
    set(counts['event_type_level0_train'].keys()))

13
13
13


set()

In [52]:
print(len(set(counts['event_type_test'].keys())))
print(len(set(counts['event_type_dev'].keys())))
print(len(set(counts['event_type_train'].keys())))
set(counts['event_type_test'].keys()).union(set(counts['event_type_dev'].keys())).difference(
    set(counts['event_type_train'].keys()))

34
35
49


{'Contact.RequestCommand.Correspondence'}

In [53]:
chosen_argument_types = set(counts['role_type_train'].keys()).intersection(set(counts['role_type_test'].keys()).union(
    set(counts['role_type_dev'].keys())))
len(chosen_argument_types)

44

In [54]:
len(set(counts['role_type_test'].keys()).union(set(counts['role_type_dev'].keys())).difference(
    set(counts['role_type_train'].keys())))

2

In [55]:

universal_coref = defaultdict(list)
universal_informative_mentions = defaultdict(list)

for coref_filename in coref_files:
    lines = open(coref_filename).readlines()
    jsons = [json.loads(line) for line in lines]
    for j in jsons:
        for list_of_ids in j['clusters']:
            for id in list_of_ids:
                for id2 in list_of_ids:
                    if id2 != id:
                        universal_coref[id].append(id2)
        universal_informative_mentions[j['doc_key']] = j['informative_mentions']

In [60]:
gtt_formats = defaultdict(list)
index = 0

event_type_to_ordering = defaultdict(list)
event_type_to_templates = defaultdict(list)


def token_index_to_text_index(tokens, token_index):
    return len(" ".join(tokens[:token_index]).replace(" \\u", "\\u"))


for file in file_names:
    for example in data[file]:
        index += 1
        new_example = {'original_doc_id': example["doc_id"], 'docid': index,
                       'doctext': " ".join(example['tokens']).replace(" \\u", "\\u"), 'token': example['tokens'],
                       'templates': []}
        gtt_formats[file].append(new_example)
        for event in example['event_mentions']:
            event_type = event['event_type'][:event['event_type'].rindex('.')]
            new_template = {"incident_type": event_type, "Trigger": event['trigger']['text'],
                            "TriggerIndex": len(
                                " ".join(example['tokens'][:event["trigger"]["start"]]).replace(" \\u", "\\u"))}
            event_type_to_templates[event_type].append(new_template)
            new_example['templates'].append(new_template)
            ordering = event_type_to_ordering[event_type]
            for role in ordering:
                new_template[role] = []

            for argument in event['arguments']:
                role = argument['role']
                if role not in new_template:
                    ordering.append(role)
                    for template in event_type_to_templates[event_type]:
                        template[role] = []
                argument_mention = [x for x in example['entity_mentions'] if x['id'] == argument['entity_id']][0]

                new_template[role].append(
                    [[argument['text'], token_index_to_text_index(example['tokens'], argument_mention['start'])]])
                pass


In [61]:
gtt_formats = dict(gtt_formats)
pass

In [62]:
for filename, records in gtt_formats.items():
    with open(filename + ".jsonl", 'w') as f:
        for record in records:
            json_str = json.dumps(record)
            f.write(json_str + '\n')


In [63]:
json.dump(dict(event_type_to_ordering), open("EventTypeToRoles.json", "w+"))